In [0]:
!pip install -q seaborn

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import math
import torch.utils.data as Data
from torch.autograd import Variable
import tensorflow as tf

In [0]:
import math
import matplotlib.pyplot as plt

In [0]:
df = pd.read_csv("dataset_18065.csv")
df.head()
tiers = df.Tier
low_df = df[df['Tier'] == '0']
mid_df = df[df['Tier'] == '1']
high_df = df[df['Tier'] == '2']
features = ['year', 'VAA', 'PSNV', 'GE', 'RQ', 'RL', 'CC']

for i in low_df.index:
  for f in features:
    low_df.loc[i, f] = low_df.loc[i, f].astype(float)
  try:
    low_df.loc[i, 'GDPgrowth'] = float(low_df.loc[i, 'GDPgrowth'])
  except ValueError:
    low_df.loc[i, 'GDPgrowth'] = float('nan')


print(low_df.GDPgrowth)

for i in mid_df.index:
  for f in features:
    mid_df.loc[i, f] = mid_df.loc[i, f].astype(float)
  try:
    mid_df.loc[i, 'GDPgrowth'] = float(mid_df.loc[i, 'GDPgrowth'])
  except ValueError:
    mid_df.loc[i, 'GDPgrowth'] = float('nan')
  

for i in high_df.index:
  for f in features:
    high_df.loc[i, f] = high_df.loc[i, f].astype(float)
  try:
    high_df.loc[i, 'GDPgrowth'] = float(high_df.loc[i, 'GDPgrowth'])
  except ValueError:
    high_df.loc[i, 'GDPgrowth'] = 0.0





/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


1           NaN
11      4.44652
13      4.64447
14        4.353
29      3.61654
         ...   
3456    5.12212
3463   -2.70147
3466    5.75793
3467     3.7949
3468    6.15919
Name: GDPgrowth, Length: 925, dtype: object


In [0]:
# cleaning data
df = pd.read_csv("dataset_18065.csv")
df = df.drop(['country', 'country_abb'], axis=1)
drop_row_indices = []
for i, row in df.iterrows():
    for j, col in row.iteritems():
        try:
            row[j] = float(col)
        except ValueError:
            # print("ValueError")
            drop_row_indices.append(i)
            continue
        if math.isnan(row[j]):
            drop_row_indices.append(i)
df = df.drop(drop_row_indices, axis = 0)
GDP_growth=df['GDPgrowth']

tiers = df.Tier
low_df = df[df['Tier'] == '0']
mid_df = df[df['Tier'] == '1']
high_df = df[df['Tier'] == '2']
features = ['year', 'VAA', 'PSNV', 'GE', 'RQ', 'RL', 'CC']


# making tiers indicator variables
df['Tier'] = df['Tier'].map({'0': 'Low', '1': 'Mid', '2': 'High'})
df = pd.get_dummies(df, prefix_sep='')
df = df.drop(labels=df.columns[10:], axis=1)
df = pd.concat([df, GDP_growth], axis=1)


In [0]:
features = ['year', 'VAA', 'PSNV', 'GE', 'RQ', 'RL', 'CC']

In [0]:
# Device configuration


In [0]:
# input_size = len(features)
# hidden_size = input_size
# num_classes = 1 # size of output

low_tier_X = low_df.loc[:, features]
low_tier_y = low_df.GDPgrowth

# print(len(low_tier_X))

mid_tier_X = mid_df.loc[:, features]
mid_tier_y = mid_df.GDPgrowth

# print(len(mid_tier_X))

high_tier_X = high_df.loc[:, features]
high_tier_y = high_df.GDPgrowth

low_X_train, low_X_test, low_y_train, low_y_test = train_test_split(np.asarray(low_tier_X).astype(np.float32),
                                                                    np.asarray(low_tier_y).astype(np.float32), test_size = .15)

mid_X_train, mid_X_test, mid_y_train, mid_y_test = train_test_split(np.asarray(mid_tier_X).astype(np.float32),
                                                                    np.asarray(mid_tier_y).astype(np.float32), test_size = .15)

high_X_train, high_X_test, high_y_train, high_y_test = train_test_split(np.asarray(high_tier_X).astype(np.float32),
                                                                        np.asarray(high_tier_y).astype(np.float32), test_size = .15)


# X_train, X_test, y_train, y_test = train_test_split(np.asarray(X).astype(np.float32), 
#                                                     np.asarray(y).astype(float32), test_size = .15)
# X_train = torch.tensor(X_train)
# X_test = torch.tensor(X_test)
# y_train = torch.tensor(y_train)
# y_test = torch.tensor(y_test)

In [0]:
low_X_train, low_X_test = torch.tensor(low_X_train), torch.tensor(low_X_test)
low_y_train, low_y_test = torch.tensor(low_y_train), torch.tensor(low_y_test)

mid_X_train, mid_X_test = torch.tensor(mid_X_train), torch.tensor(mid_X_test)
mid_y_train, mid_y_test = torch.tensor(mid_y_train), torch.tensor(mid_y_test)

high_X_train, high_X_test = torch.tensor(high_X_train), torch.tensor(high_X_test)
high_y_train, high_y_test = torch.tensor(high_y_train), torch.tensor(high_y_test)

In [0]:
low_train_dataset = Data.TensorDataset(low_X_train, low_y_train)
low_test_dataset = Data.TensorDataset(low_X_test, low_y_test)

mid_train_dataset = Data.TensorDataset(mid_X_train, mid_y_train)
mid_test_dataset = Data.TensorDataset(mid_X_test, mid_y_test)

high_train_dataset = Data.TensorDataset(high_X_train, high_y_train)
high_test_dataset = Data.TensorDataset(high_X_test, high_y_test)

In [0]:
# Hyper-parameters 
input_size = len(features)
hidden_size = len(features)
num_classes = 1
num_epochs = 100
batch_size = 200
learning_rate = 0.8

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        #self.linear = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        #out = self.linear(out)
        out = self.fc2(out)
        return out


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, num_classes).to(device)

# Loss and optimizer
loss_func = torch.nn.MSELoss()  # this is for regression mean squared loss
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)


# Data loader
train_loader = torch.utils.data.DataLoader(dataset=high_train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)


test_loader = torch.utils.data.DataLoader(dataset=high_test_dataset, 
                                          batch_size=1, 
                                          shuffle=False)

In [0]:

#print("hi!)")
train_loss = 0
#print("train_loss: ", train_loss)
for epoch in range(num_epochs):
  #print("in for loop")
  for entry in enumerate(train_loader): # for each training step
    batch_x = entry[1][0]
    batch_y = entry[1][1]
    # Converting to correct data type
    b_x = Variable(batch_x)
    #print(b_x)
    b_y = Variable(batch_y)

    #forward pass
    ypred = model(batch_x)
    #print(ypred)
    loss = loss_func(ypred,b_y)
    #print("loss: ", loss)
    train_loss += loss

    #Backpropogation 
    optimizer.zero_grad() 
    loss.backward()
    optimizer.step()

#print("train_loss = ", train_loss/(num_epochs * rounds))

# total_loss = 0
# #rounds = 0
# for epoch in range(num_epochs):
#   rounds = 0
#   for entry in enumerate(test_loader): # for each training step
#     #print(entry)
#     #Move tensors to the configured device
#     batch_x = entry[1][0]
#     batch_y = entry[1][1]
#     b_x = Variable(batch_x)
#     b_y = Variable(batch_y)

#     rounds += 1
#     #forward pass
#     ypred = model(batch_x)
#     #print(ypred)
#     loss = loss_func(ypred,batch_y)
#     total_loss+=loss
#     #print(loss)
# av_loss = total_loss/(num_epochs*rounds)

rounds=0
total_loss = 0
for step, (batch_x,batch_y) in enumerate(test_loader): # for each training step
  rounds+=1
  #forward pass
  ypred = model(batch_x)
  loss = loss_func(ypred,batch_y)
  total_loss +=loss
  
av_loss = total_loss/rounds

/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/loss.py:432: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, 

In [0]:
print(av_loss)

tensor(14.7029, grad_fn=<DivBackward0>)


In [0]:
print(loss)

tensor(2.2138, grad_fn=<MseLossBackward>)


In [0]:
'''
high DS
lr = .15, batch size 10, error = 14.8162
batch size 100, error = 14.7104
batch size 200, error = 14.84

lr = .5, batch size = 10, error = 15.4074
batch size = 100, error = 14.66
batch size = 200, error = 14.74

lr = .8, batch size = 10, error = 15.00
batch size 100, error = 14.81
batch size 200, error = 14.70
__________________

lr = .15 batch size 10: error = 19.0653
100, error = 18.0683 
200, error = 18.1770

lr = .5
batch size 200, error = 18.3277
batch size 100, error = 20.4618
batch size 10, error = 18.3074

lr = .8
batch size 10, error = 18.154
batch size 100, error = 18.2751
batch size 200, error = 18.3092
__________

lr = .15, batch size 10, error = 17.42
batch size 100, error = 16.98
batch size 200, error = 16.96

lr = .5, batch size 10, error = 17.77
batch size 100, error = 18.94
batch size 200, error = 22.67

lr = .8, batch size = 10, error = 20.3421
batch size = 100, error = 24.10
batch size = 200, error = 18.12
'''